In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# === 1. Подготовка данных ===
df = pd.read_csv("merged_freight_oil.csv", parse_dates=["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# Только числовые данные
data = df[["Freight_Price", "Oil_Price"]].values.astype(np.float32)

# Масштабируем
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# === 2. Формируем временные последовательности ===
def create_sequences(data, seq_len):
    xs, ys = [], []
    for i in range(len(data) - seq_len):
        x = data[i:i+seq_len]
        y = data[i+seq_len, 0]  # предсказываем Freight_Price
        xs.append(x)
        ys.append(y)
    return torch.tensor(xs), torch.tensor(ys)

seq_len = 12
X, y = create_sequences(data_scaled, seq_len)

# Трен / тест
train_size = int(0.8 * len(X))
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# === 3. Модель LSTM ===
class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1])
        return out.squeeze()

model = LSTMModel()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# === 4. Обучение ===
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = loss_fn(output, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# === 5. Прогноз и визуализация ===
model.eval()
with torch.no_grad():
    y_pred = model(X_test)

y_test_inv = scaler.inverse_transform(np.hstack((y_test.unsqueeze(1), X_test[:, -1, 1].unsqueeze(1))))[:, 0]
y_pred_inv = scaler.inverse_transform(np.hstack((y_pred.unsqueeze(1), X_test[:, -1, 1].unsqueeze(1))))[:, 0]

plt.figure(figsize=(12, 6))
plt.plot(y_test_inv, label="Real")
plt.plot(y_pred_inv, label="Predicted")
plt.title("LSTM Freight Price Forecast")
plt.legend()
plt.grid(True)
plt.show()